In [90]:
import random
from numpy import pi
from qiskit import *
from qiskit.tools.visualization import plot_bloch_multivector

In [116]:
# Definimos variables
bits_Alice = []
bases_Alice = []
bases_Espia = []
bases_comun_espia = []
bits_alice = []
bits_Bob = []
resultado_espia = []
clave_espia = ""
aciertos_espia = 0

# Definimos la cantidad de bits a usar
num_bits = 16

# Función para transmitir Qubits
def transmitirQubits(bits, base):
    txQ = []
    for i in range(len(bits)):
        mycircuit = QuantumCircuit(1, 1)
        if bits[i] == '1':
            mycircuit.x(0)  # Aplicar la compuerta X para cambiar ket1 a |0⟩
        if base[i] == 'X':
            mycircuit.h(0)  # Aplicar la compuerta Hadamard para cambiar la base
        txQ.append(mycircuit)
    return txQ

# Función para medir (recibir) Qubits
def medirQubits(qubits, bases):
    rxQ = []
    for i in range(len(bases)):
        qubit = qubits[i]
        if bases[i] == 'Z':
            qubit.measure(0, 0)  # Medir en la base Z
        else:
            qubit.h(0)  # Aplicar la compuerta Hadamard para cambiar la base
            qubit.measure(0, 0)  # Medir en la base Hadamard
        job = execute(qubit, Aer.get_backend('qasm_simulator'), shots=1)
        result = job.result().get_counts(qubit)
        rxQ.append(list(result.keys())[0])  # Almacenar directamente el resultado de la medición
    return rxQ

# Lado de Alice: Generamos los bits aleatorios
for i in range(num_bits):
    bases_Alice.append(random.choice(['X', 'Z']))
    bits_Alice.append(random.choice(['0', '1']))
    bases_Espia.append(random.choice(['X', 'Z']))

# Transmitir qubits de Alice a Bob
tx_Alice = transmitirQubits(bits_Alice, bases_Alice)


In [117]:
print("\n########## EVE realiza un Ataque de Interceptación y Reenvío ##########", "\n")
print(f"{'Base Alice': <15} {'Base Espía': <15} {'Base común': <15} {'Bits Alice': <15} {'Bits Espía': <15}")

for i in range(num_bits):
    # Si las bases coinciden, se compara y se forma la clave
    if bases_Alice[i] == bases_Espia[i]:
        bases_comun_espia.append(bases_Alice[i])
        resultado_espia.append(bits_Alice[i])
        clave_espia += bits_Alice[i]
        aciertos_espia += 1
    else:
        bases_comun_espia.append("-")  # La base común no coincide
        # Si la base de Alice coincide con la del Espía, se muestra la discrepancia
        if bases_Alice[i] == bases_Espia[i]:
            aciertos_espia += 1
            resultado_espia.append('-')
        else:
            # Si las bases no coinciden, se simula la intervención del espía
            if random.choice(['a', 'b']) == 'a':
                # El espía elige estado 'a' (0 o positivo)
                resultado_espia.append('0')
            else:
                # El espía elige estado 'b' (1 o negativo)
                resultado_espia.append('1')

        clave_espia += resultado_espia[i]

    # Almacenar los bits de Alice en la variable bits_alice
    bits_alice.append(bits_Alice[i])

    print(f"{bases_Alice[i]: <15} {bases_Espia[i]: <15} {bases_comun_espia[i]: <15} {bits_Alice[i]: <15} {resultado_espia[i]: <15}")

# Calcular el porcentaje de aciertos entre Alice y el Espía
porcentaje_aciertos_espia = (aciertos_espia / num_bits) * 100
print("\nBits Interceptados por el Espía: ", clave_espia)
print("Porcentaje de aciertos en las bases de Alice con las bases del Espía:", porcentaje_aciertos_espia, "%", "\n")


########## EVE realiza un Ataque de Interceptación y Reenvío ########## 

Base Alice      Base Espía      Base común      Bits Alice      Bits Espía     
X               X               X               0               0              
Z               X               -               1               1              
Z               X               -               1               0              
Z               Z               Z               0               0              
Z               Z               Z               1               1              
Z               X               -               1               1              
X               X               X               0               0              
X               X               X               0               0              
Z               Z               Z               1               1              
X               X               X               1               1              
X               Z               -            

In [120]:
# Definir función random para Bob
def randomBob():
    return random.choice(['0', '1'])

aciertos_alice_espia_bob = 0
clave_alice_espia_bob = ""

aciertos_bases_AB = 0
aciertos_bits_AB = 0

# Lista para almacenar los bits de Bob
bits_bob = []

print("\n########## Comparación de Bases entre Alice, Bob y Espía ##########", "\n")
print(f"{'Bases Alice': <15} {'Bases Espía': <15} {'Bases Bob': <15} {'Bits Alice': <15} {'Bits Bob': <15}")

# Bucle de comparación entre Alice, Espía y Bob
for i in range(num_bits):
    # Comparar la base de Alice con la base de Bob
    if bases_Alice[i] == bases_Bob[i]:
        # Bases coinciden entre Alice y Bob
        aciertos_bases_AB += 1
        # Bases coinciden entre Alice y Bob
        resultado_alice_bob = bits_Alice[i]  # Mostrar el bit de Alice en Bits Bob
    else:
        # Bases no coinciden entre Alice y Bob
        resultado_alice_bob = "-"

    # Comparar la base de Alice, Espía y Bob
    if bases_Alice[i] == bases_Bob[i] != bases_Espia[i]:
        # La base de Alice coincide con la de Bob pero no con la de Espía
        resultado_alice_bob = random.choice(['0', '1'])  # Ejecutar el random de Bob
        bits_bob.append(resultado_alice_bob)  # Agregar el bit a la lista de bits de Bob
    else:
        bits_bob.append(resultado_alice_bob)  # Si no, colocar el resultado de la comparación anterior

    print(f"{bases_Alice[i]: <15} {bases_Espia[i]: <15} {bases_Bob[i]: <15} {bits_Alice[i]: <15} {bits_bob[i]: <15}")

# Se calculan el porcentaje de consistencia entre las bases y bits de Alice y Bob
# Imprimir los bits de Alice y Bob al final
#print("\nBits de Alice:", " ".join(bits_alice))
#print("Bits de Bob:  ", " ".join(bits_bob))

# Contador para las coincidencias
coincidencias = 0

# Verificar la coincidencia en cada posición
for bit_alice, bit_bob in zip(bits_alice, bits_bob):
    # Ignorar los guiones
    if bit_bob != '-':
        # Verificar la coincidencia
        if bit_alice == bit_bob:
            coincidencias += 1

# Imprimir el resultado
print(f"\nCantidad de Bits coincidentes entre Alice y Bob: {coincidencias}")
print(f"Cantidad de bases Alice-Bob: {aciertos_bases_AB}")
print(f"\nPorcentaje de consistencia entre Bases codificación: {(coincidencias / aciertos_bases_AB) * 100}%")


########## Comparación de Bases entre Alice, Bob y Espía ########## 

Bases Alice     Bases Espía     Bases Bob       Bits Alice      Bits Bob       
X               X               Z               0               -              
Z               X               X               1               -              
Z               X               X               1               -              
Z               Z               Z               0               0              
Z               Z               Z               1               1              
Z               X               X               1               -              
X               X               Z               0               -              
X               X               Z               0               -              
Z               Z               X               1               -              
X               X               Z               1               -              
X               Z               X               1